# Day 7
## Part 1

In [1]:
type HandType =
    | FiveOfAKind  = 6
    | FourOfAKind  = 5
    | FullHouse    = 4
    | ThreeOfAKind = 3
    | TwoPair      = 2
    | OnePair      = 1
    | HighCard     = 0

let orderedCards = "23456789TJQKA" |> Seq.toArray
let scoreCard (card : char) =
    Array.findIndex ((=) card) orderedCards

let handType (hand:string) =
    let distinctCards = hand |> Seq.countBy id |> List.ofSeq |> List.map snd |> List.sortDescending
    match distinctCards with
    | [5]         -> HandType.FiveOfAKind
    | [4;1]       -> HandType.FourOfAKind
    | [3;2]       -> HandType.FullHouse
    | [3;1;1]     -> HandType.ThreeOfAKind
    | [2;2;1]     -> HandType.TwoPair
    | [2;1;1;1]   -> HandType.OnePair
    | [1;1;1;1;1] -> HandType.HighCard
    | _           -> raise (ArgumentException "Invalid hand")

let handScore hand =
    (handType hand), (hand |> Seq.map scoreCard |> List.ofSeq)


In [2]:
#r "nuget: FsUnit.xUnit"

open FsUnitTyped

handType "AAAAA" |> shouldEqual HandType.FiveOfAKind
handType "AA8AA" |> shouldEqual HandType.FourOfAKind
handType "23332" |> shouldEqual HandType.FullHouse
handType "TTT98" |> shouldEqual HandType.ThreeOfAKind
handType "23432" |> shouldEqual HandType.TwoPair
handType "A23A4" |> shouldEqual HandType.OnePair
handType "23456" |> shouldEqual HandType.HighCard

"23456789TJQKA" |> Seq.map scoreCard |> List.ofSeq |> shouldEqual [ 0; 1; 2; 3; 4; 5; 6; 7; 8; 9; 10; 11; 12 ]

let testHands = 
    [
        "32T3K"
        "T55J5"
        "KK677"
        "KTJJT"
        "QQQJA"
    ]

testHands |> List.sortByDescending handScore |> shouldEqual ["QQQJA"; "T55J5"; "KK677"; "KTJJT"; "32T3K"]



Installed Packages FsUnit.xUnit, 5.6.1

In [3]:
let scoreHands (hands:(string*int)[]) =
    hands
    |> Array.sortBy (fst >> handScore)
    |> Array.mapi (fun i (hand, bid) -> (i+1)*bid)
    |> Array.sum

let parseHand (hand:string) =
    let bits = hand.Split(' ')
    (bits[0], int bits[1])

In [4]:
[|
    "32T3K", 765
    "T55J5", 684
    "KK677", 28
    "KTJJT", 220
    "QQQJA", 483
|]
|> scoreHands |> shouldEqual 6440


In [5]:
open System.IO

let input = File.ReadAllLines("input_07.txt")

let result1 = 
    input 
    |> Array.map parseHand 
    |> scoreHands
    

In [6]:
printfn "Result 1: %d" result1
result1 |> shouldEqual 246409899 // In case I break it

Result 1: 246409899


## Part 2

In [7]:
let orderedCards2 = "J23456789TQKA" |> Seq.toArray
let scoreCard2 (card : char) =
    Array.findIndex ((=) card) orderedCards2

let handType2 (hand:string) =
    let cardCounts = hand |> Seq.countBy id |> List.ofSeq
    let jokers = 
        cardCounts 
        |> List.tryFind (fun (card, count) -> card = 'J') 
        |> Option.map snd 
        |> Option.defaultValue 0
    let distinctCards =  cardCounts |> List.map snd |> List.sortDescending
    match distinctCards, jokers with
    | [5],_         -> HandType.FiveOfAKind
    | [4;1],0       -> HandType.FourOfAKind
    | [4;1],1       -> HandType.FiveOfAKind
    | [4;1],4       -> HandType.FiveOfAKind
    | [4;1],_       -> failwith "Invalid hand"
    | [3;2],0       -> HandType.FullHouse
    | [3;2],2       -> HandType.FiveOfAKind
    | [3;2],3       -> HandType.FiveOfAKind
    | [3;2],_       -> failwith "Invalid hand"
    | [3;1;1],0     -> HandType.ThreeOfAKind
    | [3;1;1],1     -> HandType.FourOfAKind
    | [3;1;1],3     -> HandType.FourOfAKind
    | [3;1;1],_     -> failwith "Invalid hand"
    | [2;2;1],0     -> HandType.TwoPair
    | [2;2;1],1     -> HandType.FullHouse
    | [2;2;1],2     -> HandType.FourOfAKind
    | [2;2;1],_     -> failwith "Invalid hand"
    | [2;1;1;1],0   -> HandType.OnePair
    | [2;1;1;1],1   -> HandType.ThreeOfAKind
    | [2;1;1;1],2   -> HandType.ThreeOfAKind
    | [2;1;1;1],_   -> failwith "Invalid hand"
    | [1;1;1;1;1],0 -> HandType.HighCard
    | [1;1;1;1;1],1 -> HandType.OnePair
    | _,_           -> failwith "Invalid hand"

let handScore2 hand =
    (handType2 hand), (hand |> Seq.map scoreCard2 |> List.ofSeq)   

let scoreHands2 (hands:(string*int)[]) =
    hands
    |> Array.sortBy (fst >> handScore2)
    |> Array.mapi (fun i (hand, bid) -> (i+1)*bid)
    |> Array.sum     

In [9]:
[|
    "32T3K", 765
    "T55J5", 684
    "KK677", 28
    "KTJJT", 220
    "QQQJA", 483
|]
|> scoreHands2 |> shouldEqual 5905

In [10]:
let result2 = 
    input 
    |> Array.map parseHand 
    |> scoreHands2

printfn "Result 2: %d" result2
result2 |> shouldEqual 244848487 // In case I break it

Result 2: 244848487
